# Exercise 2.2 Sourcing weather data with API

### 1. Importing libraries

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import requests
import json
import os, glob
import re
from datetime import datetime
from pathlib import Path
import glob

In [2]:
# Creating path for easier import
data_folder = Path(r"C:\Users\faisa\Desktop\Data analysis\Python Specialization\Achievement 2\02 Data\Original Data")

# List CSV files to double-check
files = sorted(data_folder.glob("*.csv"))
print("CSV files found:")
for f in files:
    print("-", f.name)

CSV files found:
- JC-201509-citibike-tripdata.csv
- JC-201510-citibike-tripdata.csv
- JC-201511-citibike-tripdata.csv
- JC-201512-citibike-tripdata.csv
- JC-201604-citibike-tripdata.csv
- JC-201605-citibike-tripdata.csv
- JC-201606-citibike-tripdata.csv
- JC-201607-citibike-tripdata.csv
- JC-201608-citibike-tripdata.csv
- JC-201609-citibike-tripdata.csv
- JC-20161-citibike-tripdata.csv
- JC-201610-citibike-tripdata.csv
- JC-201611-citibike-tripdata.csv
- JC-201612-citibike-tripdata.csv
- JC-20162-citibike-tripdata.csv
- JC-20163-citibike-tripdata.csv
- JC-201701-citibike-tripdata.csv
- JC-201702-citibike-tripdata.csv
- JC-201703-citibike-tripdata.csv
- JC-201704-citibike-tripdata.csv
- JC-201705-citibike-tripdata.csv
- JC-201706-citibike-tripdata.csv
- JC-201707-citibike-tripdata.csv
- JC-201708 citibike-tripdata.csv
- JC-201709-citibike-tripdata.csv
- JC-201710-citibike-tripdata.csv
- JC-201711-citibike-tripdata.csv
- JC-201712-citibike-tripdata.csv
- JC-201801-citibike-tripdata.csv


In [3]:
print(f"Found {len(files)} files")


Found 118 files


In [4]:
# Mapping of all column variations to unified names
col_map = {
    "tripduration": "trip_duration",
    "Trip Duration": "trip_duration",
    
    "starttime": "start_time",
    "Start Time": "start_time",
    "started_at": "start_time",
    
    "stoptime": "stop_time",
    "Stop Time": "stop_time",
    "ended_at": "stop_time",
    
    "start station id": "start_station_id",
    "Start Station ID": "start_station_id",
    "start_station_id": "start_station_id",
    
    "start station name": "start_station_name",
    "Start Station Name": "start_station_name",
    "start_station_name": "start_station_name",
    
    "start station latitude": "start_lat",
    "Start Station Latitude": "start_lat",
    "start_lat": "start_lat",
    
    "start station longitude": "start_lng",
    "Start Station Longitude": "start_lng",
    "start_lng": "start_lng",
    
    "end station id": "end_station_id",
    "End Station ID": "end_station_id",
    "end_station_id": "end_station_id",
    
    "end station name": "end_station_name",
    "End Station Name": "end_station_name",
    "end_station_name": "end_station_name",
    
    "end station latitude": "end_lat",
    "End Station Latitude": "end_lat",
    "end_lat": "end_lat",
    
    "end station longitude": "end_lng",
    "End Station Longitude": "end_lng",
    "end_lng": "end_lng",
    
    "bikeid": "bike_id",
    "Bike ID": "bike_id",
    "ride_id": "bike_id",
    
    "usertype": "user_type",
    "User Type": "user_type",
    "member_casual": "user_type",
    
    "birth year": "birth_year",
    "Birth Year": "birth_year",
    
    "gender": "gender",
    "Gender": "gender",
    
    "rideable_type": "rideable_type"
}

# Desired final column order
final_cols = [
    "trip_duration","start_time","stop_time",
    "start_station_id","start_station_name","start_lat","start_lng",
    "end_station_id","end_station_name","end_lat","end_lng",
    "bike_id","user_type","birth_year","gender","rideable_type"
]


In [5]:
all_dfs = []

for f in files:
    try:
        df = pd.read_csv(f, low_memory=False)
        
        # Standardize column names (strip + lowercase) and map to unified names
        df = df.rename(columns={c: col_map.get(c.strip(), c.strip()) for c in df.columns})
        
        all_dfs.append(df)
        print(f"Loaded {os.path.basename(f)} with {len(df)} rows")
    except Exception as e:
        print(f"Error reading {f}: {e}")

# Combine all DataFrames
combined = pd.concat(all_dfs, ignore_index=True)

# Add any missing columns as NaN
for col in final_cols:
    if col not in combined.columns:
        combined[col] = pd.NA

# Keep only the desired columns in order
combined = combined[final_cols]

Loaded JC-201509-citibike-tripdata.csv with 6668 rows
Loaded JC-201510-citibike-tripdata.csv with 19264 rows
Loaded JC-201511-citibike-tripdata.csv with 15113 rows
Loaded JC-201512-citibike-tripdata.csv with 11838 rows
Loaded JC-201604-citibike-tripdata.csv with 16342 rows
Loaded JC-201605-citibike-tripdata.csv with 19488 rows
Loaded JC-201606-citibike-tripdata.csv with 23947 rows
Loaded JC-201607-citibike-tripdata.csv with 24436 rows
Loaded JC-201608-citibike-tripdata.csv with 34149 rows
Loaded JC-201609-citibike-tripdata.csv with 33425 rows
Loaded JC-20161-citibike-tripdata.csv with 7479 rows
Loaded JC-201610-citibike-tripdata.csv with 29611 rows
Loaded JC-201611-citibike-tripdata.csv with 21832 rows
Loaded JC-201612-citibike-tripdata.csv with 15114 rows
Loaded JC-20162-citibike-tripdata.csv with 8250 rows
Loaded JC-20163-citibike-tripdata.csv with 13511 rows
Loaded JC-201701-citibike-tripdata.csv with 12926 rows
Loaded JC-201702-citibike-tripdata.csv with 14026 rows
Loaded JC-201703

In [6]:
combined.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bike_id,user_type,birth_year,gender,rideable_type
0,61.0,2015-09-21 14:53:16,2015-09-21 14:54:17,3185,City Hall,40.717732,-74.043845,3185,City Hall,40.717732,-74.043845,24722,Subscriber,1975.0,1.0,NaN
1,290.0,2015-09-21 14:55:59,2015-09-21 15:00:50,3183,Exchange Place,40.716247,-74.033459,3187,Warren St,40.721124,-74.038051,24388,Customer,NaN,0.0,NaN
2,786.0,2015-09-21 14:56:14,2015-09-21 15:09:21,3183,Exchange Place,40.716247,-74.033459,3183,Exchange Place,40.716247,-74.033459,24442,Subscriber,1962.0,1.0,NaN
3,477.0,2015-09-21 14:58:35,2015-09-21 15:06:32,3203,Hamilton Park,40.727596,-74.044247,3203,Hamilton Park,40.727596,-74.044247,24678,Subscriber,1977.0,2.0,NaN
4,451.0,2015-09-21 14:59:55,2015-09-21 15:07:27,3203,Hamilton Park,40.727596,-74.044247,3186,Grove St PATH,40.719586,-74.043117,24574,Subscriber,1977.0,2.0,NaN


In [7]:
# export the combined file
combined.to_csv("C:/Users/faisa/Desktop/Data analysis/Python Specialization/Achievement 2/02 Data/Updated Data/combined_df_1.csv")

Since most of the dataframe cloumns are the same, we vertically join them. however, for those columns that are different in some datasets, I renamed them to have a standard column name.

### 3. weather data using NOAA API

In [8]:
# Setting NOAA token
Token = 'yDaSPwhbytuLCfoijdjVPpgsrbEVbzCp'

In [9]:
# Get the API for Jan 1st - Dec 31st 2022

r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [10]:
# Load the api response as a json

d = json.loads(r.text)

In [11]:
d

{'metadata': {'resultset': {'offset': 1, 'count': 365, 'limit': 1000}},
 'results': [{'date': '2022-01-01T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 116},
  {'date': '2022-01-02T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 114},
  {'date': '2022-01-03T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 14},
  {'date': '2022-01-04T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': -27},
  {'date': '2022-01-05T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 32},
  {'date': '2022-01-06T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 49},
  {'date': '2022-01-07T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attribut

In [12]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [13]:
# Get only the date field from all average temperature readings

dates_temp = [item['date'] for item in avg_temps]

In [14]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [15]:
temps

[116,
 114,
 14,
 -27,
 32,
 49,
 7,
 -25,
 14,
 16,
 -54,
 -19,
 40,
 48,
 -67,
 -80,
 39,
 18,
 32,
 51,
 -60,
 -59,
 -7,
 -2,
 36,
 -23,
 -42,
 1,
 -48,
 -71,
 -34,
 -17,
 23,
 64,
 58,
 -28,
 -48,
 5,
 41,
 28,
 63,
 86,
 118,
 28,
 -43,
 -47,
 16,
 116,
 99,
 1,
 -26,
 41,
 56,
 144,
 11,
 8,
 -11,
 24,
 8,
 26,
 77,
 56,
 -19,
 31,
 96,
 182,
 87,
 32,
 40,
 72,
 51,
 -23,
 53,
 116,
 136,
 82,
 139,
 131,
 143,
 104,
 107,
 72,
 57,
 96,
 104,
 59,
 -15,
 -12,
 26,
 99,
 131,
 72,
 70,
 76,
 97,
 86,
 91,
 128,
 114,
 86,
 88,
 134,
 154,
 163,
 157,
 157,
 83,
 79,
 76,
 94,
 106,
 146,
 133,
 128,
 104,
 115,
 116,
 82,
 104,
 129,
 136,
 121,
 124,
 120,
 172,
 148,
 100,
 104,
 135,
 154,
 161,
 171,
 160,
 179,
 188,
 204,
 197,
 178,
 155,
 154,
 212,
 278,
 210,
 174,
 162,
 164,
 202,
 204,
 207,
 233,
 283,
 178,
 203,
 195,
 219,
 200,
 222,
 218,
 237,
 244,
 218,
 218,
 205,
 242,
 239,
 233,
 203,
 252,
 217,
 173,
 205,
 221,
 191,
 192,
 219,
 258,
 270,
 243,
 22

In [16]:
# Storing the results in a dataframe

df_temp = pd.DataFrame()

In [17]:
# Get only date and cast it to date time; convert temperature from tenths of Celsius to normal Celsius

df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [18]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


### 4. Merge bike trip and weather data

In [19]:
combined.dtypes

trip_duration         float64
start_time             object
stop_time              object
start_station_id       object
start_station_name     object
start_lat             float64
start_lng             float64
end_station_id         object
end_station_name       object
end_lat               float64
end_lng               float64
bike_id                object
user_type              object
birth_year            float64
gender                float64
rideable_type          object
dtype: object

In [20]:
# Convert 'start_time' from string to datetime (day comes first in the input format)

combined['start_time'] = pd.to_datetime(combined['start_time'], dayfirst = True, errors='coerce')

C:\Users\faisa\AppData\Local\Temp\ipykernel_6840\2514865294.py:3: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  combined['start_time'] = pd.to_datetime(combined['start_time'], dayfirst = True, errors='coerce')


In [21]:
# Extract only the date part (YYYY-MM-DD) from 'started_at' into a new column

combined['date'] = pd.to_datetime(combined['start_time'], format='%Y-%m-%d').dt.date

In [22]:
combined['date'].value_counts()

date
2022-09-17    4527
2022-07-04    4497
2022-09-10    4389
2022-07-09    4311
2022-08-13    4298
              ... 
2021-02-03      38
2021-02-07      32
2017-03-18      31
2021-02-01       2
2017-03-17       1
Name: count, Length: 2040, dtype: int64

In [23]:
# Convert 'date' from string to datetime

combined['date'] = pd.to_datetime(combined['date'])

In [24]:
# Merge dataframes

df_merged = combined.merge(df_temp, how = 'left', on = 'date', indicator = True)

In [25]:
# Inspecting merged df

df_merged.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bike_id,user_type,birth_year,gender,rideable_type,date,avgTemp,_merge
0,61.0,2015-09-21 14:53:16,2015-09-21 14:54:17,3185,City Hall,40.717732,-74.043845,3185,City Hall,40.717732,-74.043845,24722,Subscriber,1975.0,1.0,NaN,2015-09-21,NaN,left_only
1,290.0,2015-09-21 14:55:59,2015-09-21 15:00:50,3183,Exchange Place,40.716247,-74.033459,3187,Warren St,40.721124,-74.038051,24388,Customer,NaN,0.0,NaN,2015-09-21,NaN,left_only
2,786.0,2015-09-21 14:56:14,2015-09-21 15:09:21,3183,Exchange Place,40.716247,-74.033459,3183,Exchange Place,40.716247,-74.033459,24442,Subscriber,1962.0,1.0,NaN,2015-09-21,NaN,left_only
3,477.0,2015-09-21 14:58:35,2015-09-21 15:06:32,3203,Hamilton Park,40.727596,-74.044247,3203,Hamilton Park,40.727596,-74.044247,24678,Subscriber,1977.0,2.0,NaN,2015-09-21,NaN,left_only
4,451.0,2015-09-21 14:59:55,2015-09-21 15:07:27,3203,Hamilton Park,40.727596,-74.044247,3186,Grove St PATH,40.719586,-74.043117,24574,Subscriber,1977.0,2.0,NaN,2015-09-21,NaN,left_only


In [26]:
df_merged['_merge'].value_counts(dropna = False)

_merge
left_only     4817844
both           895485
right_only          0
Name: count, dtype: int64

Some temperatures are not there because I only got the weather data of 2022

In [ ]:
df_merged.to_csv("C:/Users/faisa/Desktop/Data analysis/Python Specialization/Achievement 2/02 Data/Updated Data/merged_df_1.csv")